In [1]:
## 2.1 Introduction<a id='2.2_Introduction'></a>

This is an original dataset, made publicly available for researchers.

We collected 60,000 Stack Overflow questions from 2016-2020 and classified them into three categories:

HQ: High-quality posts with 30+ score and without a single edit.
LQ_EDIT: Low-quality posts with a negative score and with multiple community edits. However, they still remain open after the edits.
LQ_CLOSE: Low-quality posts that were closed by the community without a single edit.
Notes:

Questions are sorted according to Question Id.
Question body is in HTML format.
All dates are in UTC format.

## 2.2 Objectives

There are some fundamental questions to resolve in this notebook before you move on.

* Do you think you may have the data you need to tackle the desired question?
    * Have you identified the required target value?
    * Do you have potentially useful features?
* Do you have any fundamental issues with the data?

## 2.3 Imports <a id='2.3_Imports'></a>

In [2]:
#Code task 1#
#Import pandas, matplotlib.pyplot, and seaborn in the correct lines below
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import nltk
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from textblob import TextBlob

## 2.4 Load The Stack Overflow Data<a id='2.5_Load_The_Stack_Overflow_Data'></a>

In [3]:
# the supplied CSV data file is the raw_data directory

# stack_data = pd.read_csv('https://www.kaggle.com/imoore/60k-stack-overflow-questions-with-quality-rate?select=data.csv', error_bad_lines=False)

stack_data = pd.read_csv('../850380_1463404_compressed_data.csv/data.csv')

In [4]:
#Code task 2#
#Call the info method on stack_data to see a summary of the data
stack_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60197 entries, 0 to 60196
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            60171 non-null  object
 1   Title         60011 non-null  object
 2   Body          59999 non-null  object
 3   Tags          59998 non-null  object
 4   CreationDate  59998 non-null  object
 5   Y             59997 non-null  object
dtypes: object(6)
memory usage: 2.8+ MB


In [5]:
#Code task 3#
#Call the head method on ski_data to print the first several rows of the data
stack_data.head()

,Id,Title,Body,Tags,CreationDate,Y
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,1/1/2016 0:21,LQ_CLOSE
1,34552974,How to get all the child records from differen...,I am having 4 different tables like \r\nselect...,<sql><sql-server>,1/1/2016 1:44,LQ_EDIT
2,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,1/1/2016 2:03,HQ
3,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,1/1/2016 2:48,HQ
4,34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",<swift><operators><whitespace><ternary-operato...,1/1/2016 3:30,HQ


In [6]:
stack_data.isnull().sum()

Id               26
Title           186
Body            198
Tags            199
CreationDate    199
Y               200
dtype: int64

In [7]:
stack_data.dropna(inplace = True)
stack_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59997 entries, 0 to 60196
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            59997 non-null  object
 1   Title         59997 non-null  object
 2   Body          59997 non-null  object
 3   Tags          59997 non-null  object
 4   CreationDate  59997 non-null  object
 5   Y             59997 non-null  object
dtypes: object(6)
memory usage: 3.2+ MB


In [8]:
stack_data['Id'].value_counts().head(25)

51290964    1
57980900    1
58631895    1
53215736    1
41150305    1
44043036    1
42768079    1
50870500    1
52764926    1
47618688    1
43976052    1
41176751    1
51907819    1
41520025    1
51644649    1
45472455    1
50870951    1
38738252    1
42006773    1
50080893    1
39478956    1
36250803    1
36017724    1
51697102    1
54913541    1
Name: Id, dtype: int64

In [9]:
stack_data[['Id', 'Title', 'Body']].nunique()

Id       59997
Title    59987
Body     59997
dtype: int64

In [10]:
stack_data['Title'].value_counts().head(8)

#NAME?                                                                      6
Regular Expression                                                          3
Regular expression                                                          3
SyntaxError: Unexpected token }                                             2
The element 'system.webServer' has invalid child element 'aspNetCore'       1
Update a variable in Python 3                                               1
canvas.mpl_connect in jupyter notebook                                      1
How to write $('.div1').height($('.div2').height()); in pure JavaScript?    1
Name: Title, dtype: int64

In [11]:
stack_data = stack_data[stack_data['Y'].notna()]

In [12]:
stack_data.isnull().sum()

Id              0
Title           0
Body            0
Tags            0
CreationDate    0
Y               0
dtype: int64

In [13]:
stack_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59997 entries, 0 to 60196
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            59997 non-null  object
 1   Title         59997 non-null  object
 2   Body          59997 non-null  object
 3   Tags          59997 non-null  object
 4   CreationDate  59997 non-null  object
 5   Y             59997 non-null  object
dtypes: object(6)
memory usage: 3.2+ MB


In [14]:
stack_data['Y'].value_counts().head(8)

HQ          20000
LQ_CLOSE    19999
LQ_EDIT     19998
Name: Y, dtype: int64

In [15]:
stack_data[stack_data['Y'].apply(lambda x:x not in ['HQ', 'LQ_CLOSE', 'LQ_EDIT'])]

,Id,Title,Body,Tags,CreationDate,Y


In [16]:
stack_data.isnull().sum()

Id              0
Title           0
Body            0
Tags            0
CreationDate    0
Y               0
dtype: int64

In [17]:
print(len(stack_data))

59997


In [18]:
stack_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59997 entries, 0 to 60196
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            59997 non-null  object
 1   Title         59997 non-null  object
 2   Body          59997 non-null  object
 3   Tags          59997 non-null  object
 4   CreationDate  59997 non-null  object
 5   Y             59997 non-null  object
dtypes: object(6)
memory usage: 3.2+ MB


In [19]:
stack_data.dropna()

,Id,Title,Body,Tags,CreationDate,Y
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,1/1/2016 0:21,LQ_CLOSE
1,34552974,How to get all the child records from differen...,I am having 4 different tables like \r\nselect...,<sql><sql-server>,1/1/2016 1:44,LQ_EDIT
2,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,1/1/2016 2:03,HQ
3,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,1/1/2016 2:48,HQ
4,34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",<swift><operators><whitespace><ternary-operato...,1/1/2016 3:30,HQ
...,...,...,...,...,...,...
60192,60467932,C++ The correct way to multiply an integer and...,<p>I try to multiply an integer by a double bu...,<c++>,2/29/2020 17:46,LQ_CLOSE
60193,60468018,How can I make a c# application outside of vis...,<p>I'm very new to programming and I'm teachin...,<c#><visual-studio>,2/29/2020 17:55,LQ_CLOSE
60194,60468378,WHY DJANGO IS SHOWING ME THIS ERROR WHEN I TRY...,*URLS.PY*\r\n //URLS.PY FILE\r\n fro...,<django><django-views><django-templates>,2/29/2020 18:35,LQ_EDIT
60195,60469392,PHP - getting the content of php page,<p>I have a controller inside which a server i...,<javascript><php><html>,2/29/2020 20:32,LQ_CLOSE


In [20]:
# remove punctuations and beautify the strings in title and body column
# stack_data['Body'] = stack_data['Body'].str.replace(r'[^\w\s]+', '')
stack_data['Body'] = [BeautifulSoup(text).get_text() for text in stack_data['Body']]
stack_data['Title'] = [BeautifulSoup(text).get_text() for text in stack_data['Title']]

In [21]:
stack_data.head()

,Id,Title,Body,Tags,CreationDate,Y
0,34552656,Java: Repeat Task Every Random Seconds,I'm already familiar with repeating tasks ever...,<java><repeat>,1/1/2016 0:21,LQ_CLOSE
1,34552974,How to get all the child records from differen...,I am having 4 different tables like \r\nselect...,<sql><sql-server>,1/1/2016 1:44,LQ_EDIT
2,34553034,Why are Java Optionals immutable?,I'd like to understand why Java 8 Optionals we...,<java><optional>,1/1/2016 2:03,HQ
3,34553174,Text Overlay Image with Darkened Opacity React...,I am attempting to overlay a title over an ima...,<javascript><image><overlay><react-native><opa...,1/1/2016 2:48,HQ
4,34553318,Why ternary operator in swift is so picky?,"The question is very simple, but I just could ...",<swift><operators><whitespace><ternary-operato...,1/1/2016 3:30,HQ


In [22]:
# stack_data['Y_clean'] = stack_data['Y'].str.replace('LQ_CLOSE', 'LQ')
# stack_data['Y_clean'] = stack_data['Y'].str.replace('LQ_EDIT', 'LQ')
stack_data['Y_clean'] = stack_data['Y'].map(lambda x: "LQ" if x in ['LQ_CLOSE', 'LQ_EDIT'] else x)
stack_data.head()

,Id,Title,Body,Tags,CreationDate,Y,Y_clean
0,34552656,Java: Repeat Task Every Random Seconds,I'm already familiar with repeating tasks ever...,<java><repeat>,1/1/2016 0:21,LQ_CLOSE,LQ
1,34552974,How to get all the child records from differen...,I am having 4 different tables like \r\nselect...,<sql><sql-server>,1/1/2016 1:44,LQ_EDIT,LQ
2,34553034,Why are Java Optionals immutable?,I'd like to understand why Java 8 Optionals we...,<java><optional>,1/1/2016 2:03,HQ,HQ
3,34553174,Text Overlay Image with Darkened Opacity React...,I am attempting to overlay a title over an ima...,<javascript><image><overlay><react-native><opa...,1/1/2016 2:48,HQ,HQ
4,34553318,Why ternary operator in swift is so picky?,"The question is very simple, but I just could ...",<swift><operators><whitespace><ternary-operato...,1/1/2016 3:30,HQ,HQ


In [23]:
# extract words from the body column
# nltk.download('punkt')
words_body = stack_data['Body'].apply(lambda text: [sent for sent in sent_tokenize(text)
                                       if any(True for w in word_tokenize(sent))])
# if w.lower() in searched_words

In [24]:
words_body

0        [I'm already familiar with repeating tasks eve...
1        [I am having 4 different tables like \r\nselec...
2        [I'd like to understand why Java 8 Optionals w...
3        [I am attempting to overlay a title over an im...
4        [The question is very simple, but I just could...
                               ...                        
60192    [I try to multiply an integer by a double but ...
60193    [I'm very new to programming and I'm teaching ...
60194    [*URLS.PY*\r\n    //URLS.PY FILE\r\n    from d...
60195    [I have a controller inside which a server is ...
60196    [So i was recently helping someone out with so...
Name: Body, Length: 59997, dtype: object

In [25]:
len(words_body)

59997

In [26]:
# count the number of words
stack_data['word_count_Body'] = stack_data['Body'].apply(lambda x: len(str(x).split(" ")))
stack_data['word_count_Title'] = stack_data['Title'].apply(lambda x: len(str(x).split(" ")))
stack_data[['Title', 'Body', 'word_count_Body', 'word_count_Title']].head()

,Title,Body,word_count_Body,word_count_Title
0,Java: Repeat Task Every Random Seconds,I'm already familiar with repeating tasks ever...,55,6
1,How to get all the child records from differen...,I am having 4 different tables like \r\nselect...,105,18
2,Why are Java Optionals immutable?,I'd like to understand why Java 8 Optionals we...,19,5
3,Text Overlay Image with Darkened Opacity React...,I am attempting to overlay a title over an ima...,1233,8
4,Why ternary operator in swift is so picky?,"The question is very simple, but I just could ...",91,8


In [27]:
# count the number of characters
stack_data['char_count_Body'] = stack_data['Body'].str.len() ## this also includes spaces
stack_data['char_count_Title'] = stack_data['Title'].str.len() ## this also includes spaces
stack_data[['Title', 'Body', 'char_count_Body', 'char_count_Title']].head()

,Title,Body,char_count_Body,char_count_Title
0,Java: Repeat Task Every Random Seconds,I'm already familiar with repeating tasks ever...,306,38
1,How to get all the child records from differen...,I am having 4 different tables like \r\nselect...,573,93
2,Why are Java Optionals immutable?,I'd like to understand why Java 8 Optionals we...,106,33
3,Text Overlay Image with Darkened Opacity React...,I am attempting to overlay a title over an ima...,4509,53
4,Why ternary operator in swift is so picky?,"The question is very simple, but I just could ...",498,42


In [28]:
# count the number of stop words
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

stack_data['stopwords_Body'] = stack_data['Body'].apply(lambda x: len([x for x in x.split() if x in stop]))
stack_data['stopwords_Title'] = stack_data['Title'].apply(lambda x: len([x for x in x.split() if x in stop]))
stack_data[['Title', 'Body', 'stopwords_Title', 'stopwords_Body']].head()

,Title,Body,stopwords_Title,stopwords_Body
0,Java: Repeat Task Every Random Seconds,I'm already familiar with repeating tasks ever...,0,18
1,How to get all the child records from differen...,I am having 4 different tables like \r\nselect...,6,47
2,Why are Java Optionals immutable?,I'd like to understand why Java 8 Optionals we...,1,8
3,Text Overlay Image with Darkened Opacity React...,I am attempting to overlay a title over an ima...,1,54
4,Why ternary operator in swift is so picky?,"The question is very simple, but I just could ...",3,31


In [29]:
# count the number of numerics
stack_data['numerics'] = stack_data['Body'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
stack_data[['Body','numerics']].head()

,Body,numerics
0,I'm already familiar with repeating tasks ever...,0
1,I am having 4 different tables like \r\nselect...,1
2,I'd like to understand why Java 8 Optionals we...,1
3,I am attempting to overlay a title over an ima...,1
4,"The question is very simple, but I just could ...",3


In [30]:
# make text to lowercase
stack_data['Body'] = stack_data['Body'].apply(lambda x: " ".join(x.lower() for x in x.split()))
stack_data['Title'] = stack_data['Title'].apply(lambda x: " ".join(x.lower() for x in x.split()))
stack_data[['Title', 'Body']].head()

,Title,Body
0,java: repeat task every random seconds,i'm already familiar with repeating tasks ever...
1,how to get all the child records from differen...,i am having 4 different tables like select * f...
2,why are java optionals immutable?,i'd like to understand why java 8 optionals we...
3,text overlay image with darkened opacity react...,i am attempting to overlay a title over an ima...
4,why ternary operator in swift is so picky?,"the question is very simple, but i just could ..."


In [31]:
# removing punctuations
stack_data['Body'] = stack_data['Body'].str.replace('[^\w\s]','')
stack_data['Title'] = stack_data['Title'].str.replace('[^\w\s]','')
stack_data[['Title', 'Body']].head()

,Title,Body
0,java repeat task every random seconds,im already familiar with repeating tasks every...
1,how to get all the child records from differen...,i am having 4 different tables like select fr...
2,why are java optionals immutable,id like to understand why java 8 optionals wer...
3,text overlay image with darkened opacity react...,i am attempting to overlay a title over an ima...
4,why ternary operator in swift is so picky,the question is very simple but i just could n...


In [32]:
# removal of stop words

from nltk.corpus import stopwords
stop = stopwords.words('english')
stack_data['Body'] = stack_data['Body'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
stack_data['Title'] = stack_data['Title'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
stack_data[['Title', 'Body']].head()

,Title,Body
0,java repeat task every random seconds,im already familiar repeating tasks every n se...
1,get child records different tables based given...,4 different tables like select system select s...
2,java optionals immutable,id like understand java 8 optionals designed i...
3,text overlay image darkened opacity react native,attempting overlay title image image darkened ...
4,ternary operator swift picky,question simple could find answer doesnt retur...


In [33]:
# common words
freq_Title = pd.Series(' '.join(stack_data['Title']).split()).value_counts()[:10]
print('Freq_Title', freq_Title)
freq_Body = pd.Series(' '.join(stack_data['Body']).split()).value_counts()[:10]
print('Freq_Body', freq_Body)


Freq_Title using      4110
c          3772
python     3182
error      3016
string     2560
array      2539
file       2448
get        2437
android    2418
code       2356
dtype: int64
Freq_Body 1         28802
code      25057
0         24958
new       23171
using     20783
int       20195
like      19858
error     19805
string    19578
public    19519
dtype: int64


In [34]:
# common words removal

freq_Title = list(freq_Title.index)
stack_data['Title'] = stack_data['Title'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_Title))
print(stack_data['Title'].head())

freq_Body = list(freq_Body.index)
stack_data['Body'] = stack_data['Body'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_Body))
print(stack_data['Body'].head())

0                java repeat task every random seconds
1    child records different tables based given par...
2                             java optionals immutable
3     text overlay image darkened opacity react native
4                         ternary operator swift picky
Name: Title, dtype: object
0    im already familiar repeating tasks every n se...
1    4 different tables select system select set se...
2    id understand java 8 optionals designed immuta...
3    attempting overlay title image image darkened ...
4    question simple could find answer doesnt retur...
Name: Body, dtype: object


In [35]:
# rare words

rare_Title = pd.Series(' '.join(stack_data['Title']).split()).value_counts()[-10:]
print(rare_Title.head())
rare_Body = pd.Series(' '.join(stack_data['Body']).split()).value_counts()[-10:]
print(rare_Body.head())

regative       1
httpobserve    1
wight          1
charcer        1
vueresource    1
dtype: int64
sub_open                                                                                   1
bamwamham                                                                                  1
comxxxbuildautomationutilscommongetcomponentnamedoexecuteinternalgetcomponentnamejava89    1
arr11235789111218                                                                          1
hop_2                                                                                      1
dtype: int64


In [36]:
# rare words removal

rare_Title = list(rare_Title.index)
stack_data['Title'] = stack_data['Title'].apply(lambda x: " ".join(x for x in x.split() if x not in rare_Title))
print(stack_data['Title'].head())

rare_Body = list(rare_Body.index)
stack_data['Body'] = stack_data['Body'].apply(lambda x: " ".join(x for x in x.split() if x not in rare_Body))
print(stack_data['Body'].head())

0                java repeat task every random seconds
1    child records different tables based given par...
2                             java optionals immutable
3     text overlay image darkened opacity react native
4                         ternary operator swift picky
Name: Title, dtype: object
0    im already familiar repeating tasks every n se...
1    4 different tables select system select set se...
2    id understand java 8 optionals designed immuta...
3    attempting overlay title image image darkened ...
4    question simple could find answer doesnt retur...
Name: Body, dtype: object


In [37]:
# Lemmatization
# Lemmatization is a more effective option than stemming because it converts the word into its root word,
#  rather than just stripping the suffices.
# It makes use of the vocabulary and does a morphological analysis to obtain the root word.
# Therefore, we usually prefer using lemmatization over stemming.

from textblob import Word
stack_data['Title'] = stack_data['Title'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
print(stack_data['Title'].head())
stack_data['Body'] = stack_data['Body'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
print(stack_data['Body'].head())

0                 java repeat task every random second
1    child record different table based given paren...
2                             java optionals immutable
3     text overlay image darkened opacity react native
4                         ternary operator swift picky
Name: Title, dtype: object
0    im already familiar repeating task every n sec...
1    4 different table select system select set sel...
2    id understand java 8 optionals designed immuta...
3    attempting overlay title image image darkened ...
4    question simple could find answer doesnt retur...
Name: Body, dtype: object


In [38]:
#  N-grams
TextBlob(stack_data['Body'][0]).ngrams(2)

[WordList(['im', 'already']),
 WordList(['already', 'familiar']),
 WordList(['familiar', 'repeating']),
 WordList(['repeating', 'task']),
 WordList(['task', 'every']),
 WordList(['every', 'n']),
 WordList(['n', 'second']),
 WordList(['second', 'javautiltimer']),
 WordList(['javautiltimer', 'javautiltimertask']),
 WordList(['javautiltimertask', 'let']),
 WordList(['let', 'say']),
 WordList(['say', 'want']),
 WordList(['want', 'print']),
 WordList(['print', 'hello']),
 WordList(['hello', 'world']),
 WordList(['world', 'console']),
 WordList(['console', 'every']),
 WordList(['every', 'random']),
 WordList(['random', 'second']),
 WordList(['second', '15']),
 WordList(['15', 'unfortunately']),
 WordList(['unfortunately', 'im']),
 WordList(['im', 'bit']),
 WordList(['bit', 'rush']),
 WordList(['rush', 'dont']),
 WordList(['dont', 'show']),
 WordList(['show', 'far']),
 WordList(['far', 'help']),
 WordList(['help', 'would']),
 WordList(['would', 'apriciated'])]

In [48]:
# term frequency
# Term frequency is simply the ratio of the count of a word present in a sentence, to the length of the sentence.

tf1 = (stack_data['Body'][0:100]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0)
tf1.columns = ['words','tf']
tf1.head(30)

# NOTE: perform it in a better way/approach by using .extend keyword. Idea-> change the column in the list of words
# and perform the above action.

every                 6.0
second                6.0
im                   32.0
let                  13.0
rush                  1.0
repeating             1.0
familiar              1.0
15                    2.0
show                  8.0
say                   8.0
far                   2.0
javautiltimertask     1.0
world                 1.0
dont                 15.0
want                 28.0
print                15.0
task                  2.0
help                 22.0
console               1.0
would                14.0
hello                 6.0
bit                   5.0
javautiltimer         1.0
unfortunately         2.0
apriciated            1.0
n                     9.0
already               5.0
random                1.0
set                  27.0
item                 10.0
dtype: float64

In [40]:
# Inverse Document Frequency
# The intuition behind inverse document frequency (IDF) is that
# a word is not of much use to us if it’s appearing in all the documents.

for i,word in enumerate(tf1['words']):
  tf1.loc[i, 'idf'] = np.log(stack_data.shape[0]/(len(stack_data[stack_data['Body'].str.contains(word)])))

tf1

,words,tf,idf
0,every,6.0,2.681602
1,second,6.0,3.001700
2,im,32.0,0.615920
3,let,13.0,1.899629
4,rush,1.0,6.827663
...,...,...,...
2546,303,1.0,5.396248
2547,file_path,1.0,7.475689
2548,selfrequestr,1.0,11.002050
2549,base_lang,1.0,11.002050


In [41]:
# Term Frequency – Inverse Document Frequency (TF-IDF)
# TF-IDF has penalized words like ‘don’t’, ‘can’t’, and ‘use’ because they are commonly occurring words.

tf1['tfidf'] = tf1['tf'] * tf1['idf']
tf1

,words,tf,idf,tfidf
0,every,6.0,2.681602,16.089610
1,second,6.0,3.001700,18.010202
2,im,32.0,0.615920,19.709444
3,let,13.0,1.899629,24.695175
4,rush,1.0,6.827663,6.827663
...,...,...,...,...
2546,303,1.0,5.396248,5.396248
2547,file_path,1.0,7.475689,7.475689
2548,selfrequestr,1.0,11.002050,11.002050
2549,base_lang,1.0,11.002050,11.002050


In [42]:
# Bag of Words
#  Bag of Words (BoW) refers to the representation of text which describes the presence of words within the text data.
# The intuition behind this is that two similar text fields will contain similar kind of words,
# and will therefore have a similar bag of words.

from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=1000, lowercase=True, ngram_range=(1,1),analyzer = "word")
stack_data_bow = bow.fit_transform(stack_data['Body'])
stack_data_bow

<59997x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 1272278 stored elements in Compressed Sparse Row format>

In [43]:
# Sentiment Analysis
# If you recall, our problem was to detect the sentiment of the tweet.
# So, before applying any ML/DL models (which can have a separate feature detecting the sentiment using the textblob library).

stack_data['Body'][:5].apply(lambda x: TextBlob(x).sentiment)

0                    (-0.08750000000000001, 0.5)
1    (-0.03571428571428571, 0.40714285714285714)
2                                     (0.0, 0.0)
3      (0.06360544217687075, 0.3096371882086168)
4     (-0.08779761904761905, 0.4467261904761905)
Name: Body, dtype: object

__A:__ Above, you can see that it returns a tuple representing polarity and subjectivity of each tweet. Here, we only extract polarity as it indicates the sentiment as value nearer to 1 means a positive sentiment and values nearer to -1 means a negative sentiment. This can also work as a feature for building a machine learning model.

In [44]:
stack_data['sentiment'] = stack_data['Body'].apply(lambda x: TextBlob(x).sentiment[0] )
stack_data[['Body','sentiment']].head()

,Body,sentiment
0,im already familiar repeating task every n sec...,-0.087500
1,4 different table select system select set sel...,-0.035714
2,id understand java 8 optionals designed immuta...,0.000000
3,attempting overlay title image image darkened ...,0.063605
4,question simple could find answer doesnt retur...,-0.087798


In [45]:
# from nltk import word_tokenize, pos_tag, ne_chunk
# from nltk import RegexpParser
# from nltk import Tree
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')

# def get_continuous_chunks(text, chunk_func=ne_chunk):
#     chunked = chunk_func(pos_tag(word_tokenize(text)))
#     continuous_chunk = []
#     current_chunk = []

#     for subtree in chunked:
#         if type(subtree) == Tree:
#             current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
#         elif current_chunk:
#             named_entity = " ".join(current_chunk)
#             if named_entity not in continuous_chunk:
#                 continuous_chunk.append(named_entity)
#                 current_chunk = []
#         else:
#             continue

#     return continuous_chunk

# df = pd.DataFrame(stack_data)

# df['Body'].apply(lambda sent:get_continuous_chunks((sent)))
#read-write from a file in python


In [46]:
noun_phrases_list = []
for i, row in stack_data.iterrows():
    text = row['Body']
    blob = TextBlob(text)
#     print(row)
    noun_phrase = blob.noun_phrases
#     print(text)
    noun_phrases_list.extend(noun_phrase)
    if i >= 5:
        break
print(noun_phrases_list)
# do pre-processing to remove stars and unwanted characters for every text


['javautiltimer javautiltimertask', 'print hello world console', 'im bit rush dont show', 'different table', 'select system', 'select set', 'select item', 'select version system id n noof', 'qill n item item n noof version system n', 'n item item n version', 'systemid retrieve record', 'version item single storedprocedure', 'immutable threadsafety', 'overlay title image image', 'opacity effect', 'dim fix look custom component article preview image row article preview component component article preview', 'touchable image', 'r feed api user keyword interest parse homejs parse db need', 'heart parse db need', 'press google news var', 'requirereactnative var view stylesheet text image touchablehighlight', 'dimension var dimension requiredimensions var window dimensionsgetwindow var imagebutton requirecommonimagebutton var keywordbox requireauthenticationonboardingkeywordbox', 'additional library moduleexports reactcreateclass onpress function trigger button', 'thispropstext property', 'so

In [50]:
df = pd.DataFrame(stack_data)
df.reset_index(inplace = True, drop = True)
df

,Id,Title,Body,Tags,CreationDate,Y,Y_clean,word_count_Body,word_count_Title,char_count_Body,char_count_Title,stopwords_Body,stopwords_Title,numerics,sentiment
0,34552656,java repeat task every random second,im already familiar repeating task every n sec...,<java><repeat>,1/1/2016 0:21,LQ_CLOSE,LQ,55,6,306,38,18,0,0,-0.087500
1,34552974,child record different table based given paren...,4 different table select system select set sel...,<sql><sql-server>,1/1/2016 1:44,LQ_EDIT,LQ,105,18,573,93,47,6,1,-0.035714
2,34553034,java optionals immutable,id understand java 8 optionals designed immuta...,<java><optional>,1/1/2016 2:03,HQ,HQ,19,5,106,33,8,1,1,0.000000
3,34553174,text overlay image darkened opacity react native,attempting overlay title image image darkened ...,<javascript><image><overlay><react-native><opa...,1/1/2016 2:48,HQ,HQ,1233,8,4509,53,54,1,1,0.063605
4,34553318,ternary operator swift picky,question simple could find answer doesnt retur...,<swift><operators><whitespace><ternary-operato...,1/1/2016 3:30,HQ,HQ,91,8,498,42,31,3,3,-0.087798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59992,60467932,correct way multiply integer double,try multiply integer double obtain wrong resul...,<c++>,2/29/2020 17:46,LQ_CLOSE,LQ,151,11,762,55,28,4,3,-0.062500
59993,60468018,make application outside visual studio,im programming im teaching made calculator cal...,<c#><visual-studio>,2/29/2020 17:55,LQ_CLOSE,LQ,108,11,558,57,55,3,0,0.187302
59994,60468378,django showing try open new page hyperlink,urlspy urlspy file djangocontrib import admin ...,<django><django-views><django-templates>,2/29/2020 18:35,LQ_EDIT,LQ,249,16,1128,78,20,0,0,0.000000
59995,60469392,php getting content php page,controller inside server connected network sea...,<javascript><php><html>,2/29/2020 20:32,LQ_CLOSE,LQ,95,8,574,37,25,2,0,0.000000


In [ ]:
# persist the data in a file (e.g. per phrase in 1 line of a file)
# what are the noun_phrases there in HQ and LQ data- put it in a file
# read naive bayes algorithm - by using count vectoriser.
# simple test classification using naive bayes algorithm and also measure its precision recall/accuraacy.

In [ ]:
df.to_csv('formatted_stack_data.csv')